In [ ]:
'''_In this script, we ..._'''

In [ ]:
# -*- coding: utf-8 -*- #
# Authors: gracecaj@stud.ntnu.no, sbaarak@stud.ntnu.no, & annakale@stud.ntnu.no
# Date: 19/09/2023

In [ ]:
#%%
#* Import the code needed
import os
from pathlib import Path

In [ ]:
#%%
#* Define paths
directory_path = Path('../../')

# Import data needed (pop, ppd, type split)

In [ ]:
pop = pd.read_excel(
    directory_path / 'data' / 'raw_data' / 'Population' / 'norway_pop_projections.xlsx',
    index_col=0)

PpD = pd.read_excel(
    directory_path / 'data' / 'raw_data' / 'Population' / 'norway_pop_projections.xlsx',
    index_col=0)

TS_cj = pd.read_excel(
    directory_path / 'data' / 'raw_data' / 'Population' / 'norway_pop_projections.xlsx',
    index_col=0)

# Import DSM model

# Use pop and PpD to calculate stock over time

# Enter stock into stock driven model to get inflows by cohort over time

# multiply inflow by type split to get inflow by type split and cohort

# enter inflow by type split and cohort into inflow driven model to get stock by cohort and type split over time